<a href="https://colab.research.google.com/github/kyunghyuncho/ammi-2019-nlp/blob/master/01-day-LM/neural_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Language Modeling

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import torchtext.data as data

from torchtext import vocab
from collections import Counter
import re
from torchtext.data import TabularDataset 

In [2]:
torch.manual_seed(1)


# Neural Language Modeling

**Key Idea:** instead of using counts of n-grams, we use a neural network to estimate the probability of a word $w_i$ to follow a sequence of words $(w_{i - n + 1}, ..., w_{i - 1})$:

$P(w_i|w_{i−1}, w_{i−2}, …, w_{i−n+1}) = f(\theta)$

where $f$ is a neural network parametrized by a set of parameters $\theta$


**Advantages of Neural LM compared to N-gram LM**

1. Generalizing Ability: words with similar meanings will have embeddings that are close for some metric
    --> mitigates the data sparsity issue in N-gram LM

2. Can capture long-term dependencies

# Data Processing

# Build the Pipeline

# Define the Model

# Define the Loss Function and Optimizer

# Train the Model

# Evaluate the Model on Test Data

# Examples & Analysis